In [32]:
# Python generators
# yield

def create_square_generator(n_elements):
    for i in range(1,n_elements+1):
        yield i*i

for i in create_square_generator(10):
    print i,

print "\n"

def create_square_generator_dic(n_elements):
    for i in range(1,n_elements+1):
        yield i, i*i

for i in create_square_generator_dic(8):
    print i,


1 4 9 16 25 36 49 64 81 100 

(1, 1) (2, 4) (3, 9) (4, 16) (5, 25) (6, 36) (7, 49) (8, 64)


In [5]:
%%file demo4.py

from mrjob.job import MRJob

class MRWordFrequencyCount(MRJob):

    def mapper(self, _, line):
        # For every line
        yield "chars", len(line)
        yield "words", len(line.split())
        yield "lines", 1

    def reducer(self, key, values):
        yield key, sum(values)


if __name__ == '__main__':
    MRWordFrequencyCount.run()

Writing demo4.py


In [4]:
!head big_1.txt

The Project Gutenberg EBook of The Adventures of Sherlock Holmes
by Sir Arthur Conan Doyle
(#15 in our series by Sir Arthur Conan Doyle)

Copyright laws are changing all over the world. Be sure to check the
copyright laws for your country before downloading or redistributing
this or any other Project Gutenberg eBook.

This header should be the first thing seen when viewing this Project
Gutenberg file.  Please do not remove it.  Do not change or edit the


In [11]:
! python demo4.py big_1.txt


no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
creating tmp directory /tmp/demo4.rhf.20150723.170505.551573
writing to /tmp/demo4.rhf.20150723.170505.551573/step-0-mapper_part-00000
Counters from step 1:
  (no counters found)
writing to /tmp/demo4.rhf.20150723.170505.551573/step-0-mapper-sorted
> sort /tmp/demo4.rhf.20150723.170505.551573/step-0-mapper_part-00000
writing to /tmp/demo4.rhf.20150723.170505.551573/step-0-reducer_part-00000
Counters from step 1:
  (no counters found)
Moving /tmp/demo4.rhf.20150723.170505.551573/step-0-reducer_part-00000 -> /tmp/demo4.rhf.20150723.170505.551573/output/part-00000
Streaming final output from /tmp/demo4.rhf.20150723.170505.551573/output
"chars"	3374163
"lines"	61042
"words"	568801
removing tmp directory /tmp/demo4.rhf.20150723.170505.551573


In [ ]:
# %load demo2.py
from mrjob.job import MRJob
import re

'''
Run as:

python demo2.py big_2.txt

Outputs words and number of occurences

"zakharchenko"	1
"zakharino"	1
"zakharych"	2
"zakret"	1
"zakuska"	1
"zaletaev"	2
"zanthoma"	5
"zapata"	1


'''

WORD_RE = re.compile(r"[\w']+")


class MRWordFreqCount(MRJob):

  def mapper(self, _, line):
    for word in WORD_RE.findall(line):
      yield word.lower(), 1

  def combiner(self, word, counts):
    yield word, sum(counts)

  def reducer(self, word, counts):
    yield word, sum(counts)


if __name__ == '__main__':
  MRWordFreqCount.run()


In [27]:
!head -100 big_1.txt | python demo2.py 

no configs found; falling back on auto-configuration
no configs found; falling back on auto-configuration
creating tmp directory /tmp/demo2.rhf.20150723.172104.348949
reading from STDIN
writing to /tmp/demo2.rhf.20150723.172104.348949/step-0-mapper_part-00000
Counters from step 1:
  (no counters found)
writing to /tmp/demo2.rhf.20150723.172104.348949/step-0-mapper-sorted
> sort /tmp/demo2.rhf.20150723.172104.348949/step-0-mapper_part-00000
writing to /tmp/demo2.rhf.20150723.172104.348949/step-0-reducer_part-00000
Counters from step 1:
  (no counters found)
Moving /tmp/demo2.rhf.20150723.172104.348949/step-0-reducer_part-00000 -> /tmp/demo2.rhf.20150723.172104.348949/output/part-00000
Streaming final output from /tmp/demo2.rhf.20150723.172104.348949/output
"12"	1
"15"	1
"1661"	1
"1888"	1
"1971"	1
"1999"	1
"2002"	1
"29"	1
"a"	33
"abandoned"	1
"abhorrent"	1
"about"	3
"absorb"	1
"accomplished"	1
"account"	1
"across"	1
"actions"	1
"active"	1
"activity"	1
"additional"	1
"adjusted"	1
"adler"	

# Run it on the Amazon S3 cloud service

## Configuration file

```cat ~/.mrjob.conf```

```yml
runners:
  emr:
    aws_access_key_id: <Your AWS access key>
    aws_secret_access_key: <Your AWS secret key>
    ec2_key_pair: <Your key_pair name>
    ec2_key_pair_file: <Location of PEM file>
    ssh_tunnel_to_job_tracker: true
    ec2_master_instance_type: c3.xlarge
    ec2_instance_type: c3.xlarge
    num_ec2_instances: 3
```

## Run the job

```bash

python word_count.py -r emr s3://cliburn-sta663/books/*txt \
    --output-dir=s3://cliburn-sta663/wc_out \
    --no-output

```